In [ ]:
%spark2.pyspark

##///노선별 실제와 예상 비행시간 차이

from pyspark.sql.functions import *
from pyspark.sql.functions import date_format
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
df=spark.read.csv("hdfs:///user/maria_dev/pro/2008resultfull.csv", header=True, inferSchema=True)

result_df = df.groupBy("Origin","Dest").agg(F.mean("ElapsedTimeDelay").alias("ETDM"))
z.show(result_df)

##//운항 노선 갯수 출발지별
result_df = df.groupBy("Origin").agg(F.count("ElapsedTimeDelay").alias("ETDC")).sort("ETDC")
result_df1 = df.groupBy("Origin").agg(F.mean("ElapsedTimeDelay").alias("ETDM")).sort("ETDM")

result_df2 = result_df.join(result_df1,result_df1["Origin"]==result_df["Origin"])

result_df2 = result_df2.sort("ETDC")
z.show(result_df2)

%spark2.pyspark
##//운항 노선 갯수,딜레이 도착지별
from pyspark.sql.functions import *
from pyspark.sql.functions import date_format
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
df=spark.read.csv("hdfs:///user/maria_dev/pro/2008resultfull.csv", header=True, inferSchema=True)

result_df = df.groupBy("Dest").agg(F.count("ElapsedTimeDelay").alias("ETDC")).sort("ETDC")
result_df1 = df.groupBy("Dest").agg(F.mean("ElapsedTimeDelay").alias("ETDM")).sort("ETDM")

result_df2 = result_df.join(result_df1,result_df1["Dest"]==result_df["Dest"])

result_df2 = result_df2.sort("ETDC")



z.show(result_df2)

#출발지 날씨event별 딜레이 평균시간
df_weather = spark.read.csv("hdfs:///user/maria_dev/pro/pre2008weather.csv", header=True, inferSchema=True) 

df_2008 = spark.read.csv("hdfs:///user/maria_dev/pro/2008resultfull.csv", header=True, inferSchema=True)
df_2007 = spark.read.csv("hdfs:///user/maria_dev/airline_data/2007.csv", header=True, inferSchema=True)
df_2006 = spark.read.csv("hdfs:///user/maria_dev/airline_data/2006.csv", header=True, inferSchema=True)
df_2005 = spark.read.csv("hdfs:///user/maria_dev/airline_data/2005.csv", header=True, inferSchema=True)
df_2004 = spark.read.csv("hdfs:///user/maria_dev/airline_data/2004.csv", header=True, inferSchema=True)
df_2003 = spark.read.csv("hdfs:///user/maria_dev/airline_data/2003.csv", header=True, inferSchema=True)
df_airline = df_2008 df_2003.union(df_2004).union(df_2005).union(df_2006).union(df_2007).union(df_2008)

df_airline = df_airline.filter(df_airline["Cancelled"]==0).filter(df_airline["Diverted"]==0)
df_airline = df_airline.replace(["NA"], ["0"], "CarrierDelay").replace(["NA"], ["0"], "WeatherDelay")
df_airline = df_airline.replace(["NA"], ["0"], "NASDelay").replace(["NA"], ["0"], "SecurityDelay").replace(["NA"], ["0"], "LateAircraftDelay")
df_airline = df_airline.withColumn("ElapsedTime", df_airline.ArrTime - df_airline.DepTime)
df_airline = df_airline.withColumn("CRSElapsedTime", df_airline.CRSArrTime - df_airline.CRSDepTime)
df_airline = df_airline.withColumn("ElapsedTimeDelay", df_airline.CRSElapsedTime - df_airline.ActualElapsedTime)
df_airline = df_airline.drop("DayOfWeek", "FlightNum", "Distance", "AirTime", "TaxiIn", "TaxiOut", "CancellationCode")							

airport_dep = df_airline.groupby("Origin").agg(count("Year").alias("origin_cnt")).sort(desc("origin_cnt")).limit(10)
airport_arr = df_airline.groupby("Dest").agg(count("Year").alias("dest_cnt")).sort(desc("dest_cnt")).limit(10)
airport_list = [row.Origin for row in airport_dep.collect()]
df_airline = df_airline.filter(df_airline["Origin"].isin(airport_list) & df_airline["Dest"].isin(airport_list))

df_weather = df_weather.select("Year", "Month", "DayOfMonth", "AirportCode", split(df_weather["Weather"],",").alias("WeatherArray"))
df_weather = df_weather.select("Year", "Month", "DayOfMonth", "AirportCode", explode("WeatherArray").alias("Weather"))

df = df_airline.join(df_weather, (df_weather["Year"]==df_airline["Year"]) & (df_weather["Month"]==df_airline["Month"]) 
    & (df_weather["DayOfMonth"]==df_airline["DayOfMonth"])).filter(df_airline["WeatherDelay"] > 0).filter((df_airline["Origin"]==df_weather["AirportCode"]))
result_df = df.groupby("Weather").agg(mean("ElapsedTimeDelay").alias("ETDM"))
result_df1 = df.groupby("Weather").agg(count("Weather").alias("Count"))
result_df2 = result_df.join(result_df1,result_df1["Weather"]==result_df["Weather"])
result_df2 = result_df2.orderBy(desc("ETDM"))
z.show(result_df2)

